# Batch Data Processing 1

In [1]:
#Import requied libraries
import requests
import re
import pandas as pd
import time 
headers= {'User-Agent': 'Mozilla/5.0'}

In [2]:
#Use Python csv module to read from file and get all urls (each url is in separate line in the file) to a list and then parse.
import csv
from urllib.request  import urlopen
from bs4 import BeautifulSoup

In [ ]:
contents = []
with open('C:\\Users\\gaura\\Desktop\\Project\\e.csv','r') as csvf: # Open file in read mode
    urls = csv.reader(csvf)
    for url in urls:
        contents.append(url) # Add each url to list contents
appended_data = []
for url in contents:  # Parse through each url in the list.
    phone = urlopen(url[0]).read()
  
    #print(soup)
    soup=BeautifulSoup(phone)
    try:
        name = soup.find("h1", class_="specs-phone-name-title").get_text()
        specs = soup.find("div", id="specs-list")
        table = specs.find_all("table")
        rows = soup.find_all('tr')
    #print(rows[:10])
    except AttributeError: # catch the error
        continue
        
    file=[]
    list_rows = []
    for row in rows:
        for cells in row.find_all('td', class_='nfo'):
           
            if cells.name == 'br': 
                between_br = []
                for br in cells.findAll('br'):
                    following = br.nextSibling

                    if following == '\n':
                        continue

                    sport = following.strip()
                    score = str(following.next_element)

                    combined = ' '.join((sport, score))
                    between_br.append(combined)

                    cell='\n'.join(between_br)
                    str_cells = str(cell)
                    clean = re.compile('<.*?>')
                    clean2 = (re.sub(clean, '',str_cells))
           
                    list_rows.append(clean2)
            else:
                str_cells = str(cells)
                clean = re.compile('<.*?>')
                clean2 = (re.sub(clean, '',str_cells))
           
                list_rows.append(clean2)
        
        for c in row.find_all("td",class_="ttl"):
       
                   
            str_cells1 = str(c)
            clean1 = re.compile('<.*?>')
            clean21 = (re.sub(clean, '',str_cells1))
    
            file.append(clean21)
 # Convert into dataframe
    df2=pd.DataFrame(file)
# Take transpose of the dataframe
    df2=df2.T
    df1 = pd.DataFrame(list_rows)
    df1=df1.T
# Concatenate data
    df=pd.concat([df2, df1])
    try:
    # The code that can remove the error
        new_header = df.iloc[0]
    except IndexError: # Catch the error
        continue
    df.columns = new_header
    df= df[1:]
    df = df.loc[:,~df.columns.duplicated()]
    df['Link'] = url
    
        # Store DataFrame in list
    appended_data.append(df)
        
appended_data = pd.concat(appended_data,axis=0, ignore_index=True)
# Wait for 2 seconds
time.sleep(2)
           
# Data is stored in CSV file
with open('C:\\Users\\gaura\\Desktop\\Project\\data.csv', 'w', encoding='utf-8') as f:
    appended_data.to_csv(f, header=True)
